In [0]:
from pyspark.sql.functions import *
from pyspark.sql import *

### _DimUser_

In [0]:
df = spark.read.format("parquet")\
  .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/DimUser")

df.show(5)

### **_AutoLoader_**

In [0]:
df_user = spark.readStream.format("CloudFiles")\
  .option("cloudFiles.format","parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimUser/checkpoint")\
  .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/DimUser")

In [0]:
#display(df_user.limit(20))

Let do Some transformations

In [0]:
df_user = df_user.withColumn("user_name",upper(col("user_name")))

In [0]:
# display(
#     df_user.limit(20),
#     checkpointLocation="abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimUser/checkpoint_preview"
# )

Lets use utilities which we have in utils Folder

In [0]:
import os
import sys

project_path = os.path.join(os.getcwd(), "..", "..")

sys.path.append(project_path)

from utils import *


lets use our functions from utils

In [0]:
from utils.transformations import reusable

Lets Create object of function in our created util (transformation) and use it using object

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropColumns(df_user,['_rescued_data'])

#lets also drop duplicates

df_user = df_user.dropDuplicates(['user_id'])

In [0]:
#display(df_user, checkpointLocation = "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimUser/checkpoint")

Lets Write our Data

In [0]:
df_user.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option(
      "checkpointLocation",
      "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimUser/checkpoint"
  ) \
  .trigger(once=True) \
  .option("path","abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimUser/data")\
  .toTable("spotify_catalog.silver.DimUser")


-----
Now Lets Load Data For Dim Artist

## **_DimArtist_**

In [0]:
df_art = spark.readStream.format("CloudFiles")\
  .option("cloudFiles.format","parquet")\
  .option("cloudFiles.schemaLocation", "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimArt/checkpoint")\
  .option("cloudFiles.schemaEvolutionMode", "addNewColumns")\
  .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/DimArtist")

In [0]:
#display(df)

In [0]:
df_art_obj = reusable()

df_art = df_art_obj.dropColumns(df_art, ['_rescued_data'])
df_art = df_art.dropDuplicates(['artist_id'])
#display(df_art)

In [0]:
df_art.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option(
      "checkpointLocation",
      "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimArt/checkpoint"
  ) \
  .trigger(once=True) \
  .option("path","abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimArt/data")\
  .toTable("spotify_catalog.silver.DimArtist")

## **_DimTrack_**

In [0]:
df_track = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option(
        "cloudFiles.schemaLocation",
        "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimTrack/checkpoint"
    ) \
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/DimTrack")


In [0]:
df_track = df_track.withColumn("durationFlag",when(col("duration_sec")<150,"low")\
    .when(col("duration_sec")<300,"medium")\
    .otherwise("high"))

df_track = df_track.withColumn("track_name",regexp_replace(col("track_name"),'-',' '))

df_track = reusable().dropColumns(df_track,['_rescued_data'])

#display(df_track)

In [0]:
df_track.writeStream \
  .format("delta") \
  .outputMode("append") \
  .option(
      "checkpointLocation",
      "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimTrack/checkpoint"
  ) \
  .trigger(once=True) \
  .toTable("spotify_catalog.silver.DimTrack")


## **_DimDate_**

In [0]:
df_date = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option(
        "cloudFiles.schemaLocation",
        "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimDate/checkpoint"
    ) \
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/DimDate")


In [0]:
df_date = reusable().dropColumns(df_date,['_rescued_data'])

df_date.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/DimDate/checkpoint"
    ) \
    .trigger(once=True) \
    .toTable("spotify_catalog.silver.DimDate")


## _**FactStream**_

In [0]:
df_fact_stream = spark.readStream \
    .format("cloudFiles") \
    .option("cloudFiles.format", "parquet") \
    .option(
        "cloudFiles.schemaLocation",
        "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/FactStream/checkpoint"
    ) \
    .option("cloudFiles.schemaEvolutionMode", "addNewColumns") \
    .load("abfss://bronze@rkstorageprojectspotify.dfs.core.windows.net/FactStream")


In [0]:
df_fact_stream = reusable().dropColumns(df_fact_stream,['_rescued_data'])

df_fact_stream.writeStream \
    .format("delta") \
    .outputMode("append") \
    .option(
        "checkpointLocation",
        "abfss://silver@rkstorageprojectspotify.dfs.core.windows.net/FactStream/checkpoint"
    ) \
    .trigger(once=True) \
    .toTable("spotify_catalog.silver.FactStream")
